# Boole, Arithmetic and Python Syllogisms

The goal of this work is to understand how exactly Boole, in his _Mathematical Analysis of Logic_,  uses standard arithmetic to do his bidding and avoid the rough edges that arise when the analogies are stretched to their limits.
The findings are subsequently used to express the system in code and use it to solve syllogisms.

## Introduction

The variables in the algebra are the so-called _elective symbols_, which denote the operation of selecting from a universe of individuals (denoted $1$) those that satisfy some property. In the abstract formulation, the elective symbol $x$ is taken to select individuals that "are Xs".

In [1]:
# An elective symbol is just a name - it has no further properties.
Symbol = str

Conjunction is denoted by multiplication, so $xy$ selects individuals that are both Xs and Ys.
Expressions like this one formed from elective symbols are called _elective functions_.

In [2]:
from typing import List, Dict

# A mapping like {"x": 0, "y": 1} assigning the elective symbols' values.
# The evaluation will be used later.
SymbolValues = Dict[Symbol, int]

# (Elective) functions
class Function:
    # The string representation of the function
    def __repr__(self):
        raise NotImplemented

    # Returns the value of the function given the symbols' values.
    def __call__(self, values: SymbolValues):
        raise NotImplemented

        
class SymbolFunction(Function):
    def __init__(self, symbol: Symbol):
        self.symbol = symbol
    
    def __repr__(self):
        return f"{self.symbol}"
    
    def __call__(self, values: SymbolValues):
        return values[self.symbol]


class MultiplicationFunction(Function):
    def __init__(self, left: Function, right: Function):
        self.left = left
        self.right = right
    
    def __repr__(self):
        return f"{self.left}{self.right}"

    def __call__(self, values: SymbolValues):
        return self.left(values) * self.right(values)
    
xs = SymbolFunction("x")
both_xs_and_ys = MultiplicationFunction(SymbolFunction("x"), SymbolFunction("y"))

print("To select all Xs:", xs)
print("To select those that are Xs and Ys simultaneously:", both_xs_and_ys)

To select all Xs: x
To select those that are Xs and Ys simultaneously: xy


<!-- Elective symbols differ from variables in arithmetic by satisfying an innocent-looking property, the _index law_:
$
x^n = x.
$
Semantically, this means that $xx$, i.e. selecting all Xs and from those again selecting all Xs, is the same as $x$, meaning just selecting all Xs.
 -->
Boole also introduces the negation of $x$: it is denoted by $1-x$ and means "take all individuals in the universe except those that are Xs" or "not Xs" for short.
Recall that the universe is denoted by $1$ and so we could interpret the minus sign as a set or class difference.
However, Boole does not define negation this way, $1-x$ is understood rather as a unary operation on $x$, which is reflected in the code:

In [3]:
class NegationFunction(Function):
    def __init__(self, body: Function):
        self.body = body
    
    def __repr__(self):
        return f"(1-{self.body})"

    def __call__(self, values: SymbolValues):
        return 1 - self.body(values)


xs_and_not_ys = MultiplicationFunction(
    SymbolFunction("x"),
    NegationFunction(SymbolFunction("y")),
)
print("To select all Xs that are not Ys:", xs_and_not_ys)

To select all Xs that are not Ys: x(1-y)


With what was introduced so far, we have a "querying system" capable of selecting individuals with certain properties.
From a modern perspective, note that negation and conjunction together form a functionally complete set of logical connectives, meaning all other operators such as disjunction or implication can be expressed using these two.
<!-- This means, importantly, that the system would not become more expressive if we were to somehow incorporate more operators. -->

## Elective equations and 1+1=1

Some difficulties arise at the level of _elective equations_, "equations of which the members are elective functions" (p. 16).
With their introduction, we move from a querying system to a system capable of expressing propositions such as "All Xs are Ys".
This example is introduced by Boole on p. 21, where he expresses the proposition as $xy = x$.

He then directly proceeds to rewrite the equation to $x(1-y) = 0$.
This understated rewriting step is actually a large leap: we have applied arithmetic rewriting to an elective equation, which was before this point been viewed just as a sequence of symbols with no connection to arithmetic.

Boole's justification is that "all the processes of common algebra are applicable to the present system" (p. 18) because elective symbols are commutative and distributive.
We will put aside the issue that this justification is probably unsufficient, e.g. because we have not defined minus as a binary operation.
Instead, we focus on the mentioned distributive property.
The property is expressed as $x(u + v) = xu + xv$, with the specification of "$u+v$ representing the undivided subject, and $u$ and $v$ the component parts of it" (p. 17).

This is the first appearance of the plus sign in the text.
It is tempting to think that it denotes disjunction, but the situation is more complicated.
For if $+$ were simply disjunction, we would have $1+1=1$: true in Boole's logic but obviously not in arithmetic.
Rather, addition only works as disjunction when the classes are disjoint, meaning that for $u+v$ to be meaningful, $uv=0$ must hold.
This is what is meant by $u$ and $v$ being "component parts".

Yet the plus sign appears in many places throughout the text and problems of this kind are never encountered.
How so? How does Boole avoid constructing elective functions and equations that are invalid in this way while freely applying arithmetic manipulations to the equations?

The solution can be found in the above discussion on negation and conjunction being functionally complete.
The conjunction $xy$ and the negation $1-x$ impose no restrictions on $x$ and $y$, unlike $x+y$.
If we want to express disjunction while avoiding $+$, we can use the identity
$x \lor y = \lnot (\lnot x \land \lnot y)$:

In [4]:
xs_or_ys = NegationFunction(MultiplicationFunction(
    NegationFunction("x"),
    NegationFunction("y")
))
print("Disjunction on not necessarily exclusive classes:", xs_or_ys)

Disjunction on not necessarily exclusive classes: (1-(1-x)(1-y))


All other logical operators can be constructed as well.

After the intial construction, the equations are "sent to arithmetic": they are treated as simply operating on numbers, with the additional assumption of the index law holding for the elective symbols involved.
True equations will not become false through manipulations, though they might not be _interpretable_ in Boole's logic.

For instance, if $x=1$ holds (in both Boole's logic and arithmetic), then $x+1=2$ will also hold in arithmetic. However, $2$ has no meaning in Boole's logic so we cannot "go back" and interpret the equation logically.
But in some cases, like when going from $xy=x$ to $x(1-y)=0$, we can interpret the resulting equation too. Recall that $xy=x$ means "All Xs are Ys": the second equation then means "there are no individuals that are Xs and aren't Ys" where $0$ plays the role of "no individuals".

Boole's way of using arithmetic in elective equations can be summarized in the following recipe:
1. Construct an elective equation in a safe way, using conjunctions and negations.
2. Start interpreting the elective equation as an arithmetic equation.
3. Manipulate using aritmetic rules.
4. If possible, interpret the equation again as an elective one. (This may not always be possible, as in the $x+1=2$ example.)

In the rest of the text, we demonstrate a way to apply this recipe programmatically.

## Parsing interlude

For brevity in the following, a simple expression parser will be helpful.
This is really only a technical tool and can safely be skipped when reading.

In [5]:
# We could probably do without this, but it is convenient.
class ConstantFunction(Function):
    def __init__(self, value: int):
        self.value = value
    
    def __repr__(self):
        return f"{self.value}"
    
    def __call__(self, values: SymbolValues):
        return self.value

In [6]:
import re

def matching_parenthesis(s):
    count = 0
    for i, c in enumerate(s):
        if c == "(":
            count += 1
        elif c == ")":
            count -= 1
            if count == 0:
                return i

def parse(s: str):
#     print("  ", s)
    if len(s) == 1:
        match = re.fullmatch(r"([a-z])", s)
        if match:
            return SymbolFunction(match.groups()[0])

        match = re.fullmatch(r"([01])", s)
        if match:
            return ConstantFunction(int(match.groups()[0]))
        
        raise ValueError(f"Cannot parse {s}")
        
    if s[0] == "(":
        to = matching_parenthesis(s)
        assert s[0:3] == "(1-"
        
        inside = parse(s[3:to])
        res = NegationFunction(inside)
    else:
        to = 0
        res = parse(s[0:1])
    
    if to != len(s) - 1:
        right = parse(s[to+1:])
        res = MultiplicationFunction(res, right)
    
    return res

print(parse("x"))
print(parse("0"))
print(parse("(1-x)y"))

x
0
(1-x)y


## Elective equations in code

The goal now is to put down into code Boole's system of elective equations and applying it to inference in syllogisms.
However, manipulating equations programmatically or even automatically solving them is difficult.
Luckily, Boole's book itself offers a more mechanical solution in the chapter _Properties of elective functions_.
Boole notes that we can represent any elective equation (and thus proposition) of two variables as $\phi(xy) = 0$ and that this equation in turn can be written as
$$
\phi(xy) = \phi(00)(1-x)(1-y) + \phi(01)(1-x)y + \phi(10)x(1-y) + \phi(11)xy = 0
$$
(p. 62).
Of course, this decomposition generalizes naturally beyond two variables.
The terms $\phi(00), \phi(01)$ etc. are called the _moduli_ of the equation.

The specific values of the moduli are not important, it only matters whether they are zero (p. 65):
non-zero moduli determine the cases that cannot occur.
In the code we therefore normalize all non-zero moduli to $1$.
This gives us a unique normalized representation of each equation, a proto-truth table.

In [7]:
import itertools


class Equation:
    def __init__(self, symbols: List[Symbol], moduli: Dict[List[int], int]):
        self.symbols = symbols
        self.moduli = moduli

    def __repr__(self):
        terms = []
        for values, modulus in self.moduli.items():
            # If the modulus is zero, the whole term is zero.
            if modulus != 0:
                term = ""
                for (symbol, value) in zip(self.symbols, values):
                    if value == 1:
                        term += symbol
                    else:
                        term += f"(1-{symbol})"

                terms.append(term)

        if len(terms) == 0:
            return "0 = 0"
        else:
            return " + ".join(terms) + " = 0"

def create_normalized_equation(lhs: Function, rhs: Function, symbols: List[Symbol]):
#     print(f"  normalizing: {lhs} = {rhs}")
    moduli = dict()

    for values in itertools.product([0, 1], repeat=len(symbols)):
        value_map = dict(zip(symbols, values))  # e.g. {"x": True, "y": False}

        # We get the desired phi(xyz) = 0 form by subtracting the RHS.
        if lhs(value_map) - rhs(value_map) == 0:
            moduli[values] = 0
        else:
            moduli[values] = 1
    
    return Equation(symbols, moduli)

lhs = parse("x")
rhs = parse("xy")

all_xs_are_ys = create_normalized_equation(lhs, rhs, ["x", "y"])

print(f"Original equation: {lhs} = {rhs}")
print(f"Normalized version:", all_xs_are_ys)

Original equation: x = xy
Normalized version: x(1-y) = 0


Notice that the normalized version is actually the form obtained by Boole on p. 21, the example discussed before.

## Syllogisms

Boole himself chooses to represent syllogisms by eliminating a variable (the middle term) from a system of equations.
Here we present a more computer-friendly view of the problem.
We will obtain the conclusion in two steps:
1. Create a single equation expressing the conjunction of the two premises. When the equations are in their normalized form, this can be done by simply adding both together and renormalizing if needed. Since the normalized form lists the impossible cases, adding the equations performs a union of the sets of impossible cases.
2. Eliminate the middle term - to be discussed later.

This is perhaps clearer in code:

In [8]:
def equation_conjunction(a: Equation, b: Equation):
    assert a.symbols == b.symbols  # We must operate on the same symbols.
    
    moduli = dict()

    for key in a.moduli:
        if a.moduli[key] == 1 or b.moduli[key] == 1:
            moduli[key] = 1
        else:
            moduli[key] = 0
    
    return Equation(a.symbols, moduli)

all_ys_are_xs = create_normalized_equation(
    parse("y"), parse("xy"),
    ["x", "y", "z"],
)

all_zs_are_ys = create_normalized_equation(
    parse("z"), parse("yz"),
    ["x", "y", "z"],
)

conclusion = equation_conjunction(all_ys_are_xs, all_zs_are_ys)

print(
    '"All Ys are Xs" and "All Zs are Ys" together imply:',
    conclusion,
)

"All Ys are Xs" and "All Zs are Ys" together imply: (1-x)(1-y)z + (1-x)y(1-z) + (1-x)yz + x(1-y)z = 0


The expected conclusion of this _barbara_ syllogism is "All Zs are Xs", or $z(1-x) = 0$. Why did we obtain this complicated expression?

The reason is that we did not eliminate the middle term. We need an expression in which $y$ does not appear.
This amounts to determining which cases are certainly impossible, no matter the value of the middle term.
For instance, above there is a term $(1-x)y(1-z)$, but the case of $x=0$, $z=0$ still cannot be ruled out: it is not valid when $y=1$, but _is_ when $y=1$.
The only case we can rule out is $x=0$, $z=1$.

In [9]:
def eliminate_symbol(eq: Equation, symbol: Symbol):
    eliminated_count = dict()

    for values, modulus in eq.moduli.items():
        # Remove the value of the symbol being eliminated.
        values2 = list(values)
        values2.pop(eq.symbols.index(symbol))
        values2 = tuple(values2)

        if values2 not in eliminated_count:
            eliminated_count[values2] = 0
        
        if modulus == 1:
            eliminated_count[values2] += 1
    
    # For the example above, eliminated_count will now be:
    # {(0, 0): 1, (0, 1): 2, (1, 0): 1, (1, 1): 0}

    moduli = dict()
    for values, count in eliminated_count.items():
        moduli[values] = 1 if count == 2 else 0

    # We don't want the eliminated symbol in the list.
    symbols2 = eq.symbols.copy()
    symbols2.pop(eq.symbols.index(symbol))
    
    return Equation(symbols2, moduli)

print(
    "Eliminating y from the conclusion above:",
    eliminate_symbol(conclusion, "y")
)

Eliminating y from the conclusion above: (1-x)z = 0


And indeed, $(1-x)z = 0$ is the conclusion Boole reached with his method.

But this was the simplest case.
Let us now consider a more complex case of premises "All Ys are Xs" and "No Zs are Ys" (p. 35).
The expected conclusion is "Some Xs are not Zs".
If we apply our method directly, we get:

In [10]:
no_zs_are_ys = create_normalized_equation(
    parse("yz"), parse("0"),
    ["x", "y", "z"],
)

# we have `all_ys_are_xs` from before
conclusion = equation_conjunction(all_ys_are_xs, no_zs_are_ys)
print("Before elimination:", conclusion)

conclusion = eliminate_symbol(conclusion, "y")
print("After elimination:", conclusion)

Before elimination: (1-x)y(1-z) + (1-x)yz + xyz = 0
After elimination: 0 = 0


Alas, $0=0$ is a tautology so we have not learned anything. What happened?

The problem is that currently we have no way to express "Some Xs".
To fix this, we can simply use Boole's own device: write "All Ys are Xs" as $y=vx$, $v$ being an auxilliary variable that represents a part of all Xs.

In [11]:
all_ys_are_xs_v = create_normalized_equation(
    parse("y"), parse("vx"),
    ["x", "y", "z", "v"],  # We need to add the "v" variable.
)

no_zs_are_ys_v = create_normalized_equation(
    parse("yz"), parse("0"),
    ["x", "y", "z", "v"],
)

aux = create_normalized_equation(
    parse("v(1-x)"), parse("0"),
    ["x", "y", "z", "v"],
)

conclusion = equation_conjunction(all_ys_are_xs_v, no_zs_are_ys_v)
conclusion = equation_conjunction(conclusion, aux)
print("Before elimination:", conclusion)

conclusion = eliminate_symbol(conclusion, "y")
print("After elimination:", conclusion)

Before elimination: (1-x)(1-y)(1-z)v + (1-x)(1-y)zv + (1-x)y(1-z)(1-v) + (1-x)y(1-z)v + (1-x)yz(1-v) + (1-x)yzv + x(1-y)(1-z)v + x(1-y)zv + xy(1-z)(1-v) + xyz(1-v) + xyzv = 0
After elimination: (1-x)(1-z)v + (1-x)zv + xzv = 0


This is equal to Boole's own conclusion (p. 35), interpreted as "Some Xs are not Zs".
He additionally stresses that the interpretation "Some Zs are not Xs" is not possible:
"the interpretation of $vx$ is fixed, as Some Xs; $v$ is regarded as the representative of Some, only with reference to the class X."
I find this explanation confusing, but the point is that $v$ is implicitly defined a part of $x$, whereas it does not have to be fully contained in $z$.

Finally, consider a case where no inference is possible: "All Ys are Xs" and "Some Zs are not Ys" (p. 38).

In [12]:
all_ys_are_xs_v = create_normalized_equation(
    parse("y"), parse("yx"),
    ["x", "y", "z", "v"],
)

some_zs_are_not_ys_v = create_normalized_equation(
    parse("vz"), parse("v(1-y)"),
    ["x", "y", "z", "v"],
)

conclusion = equation_conjunction(all_ys_are_xs_v, some_zs_are_not_ys_v)
print("Before elimination:", conclusion)

conclusion = eliminate_symbol(conclusion, "y")
print("After elimination:", conclusion)

Before elimination: (1-x)(1-y)(1-z)v + (1-x)y(1-z)(1-v) + (1-x)y(1-z)v + (1-x)yz(1-v) + (1-x)yzv + x(1-y)(1-z)v + xyzv = 0
After elimination: (1-x)(1-z)v = 0


We again matched Boole's result. Boole additionally reduces to $0=0$ by noting that $v(1-z) = 0$, a an auxilliary equation in the usage of $v$ as "Some Zs".
This reduction is difficult to reproduce in the present system: if we add the equation $v(1-z) = 0$ to the system,
we obtain
$$
(1-x)(1-z)v + x(1-z)v = 0
$$
which is the same as $(1-z)v = 0$, the auxilliary equation itself.
We do not obtain $0=0$ because this system cannot express that $v(1-z) = 0$ is an assumption.
The result it produces is simply the strongest conclusion it can infer from the equation(s) it was given.

## Conclusion

We investigated the relationship between Boole's elective equations and standard elective equations.
The findings allowed us to implement Boole's system in Python and use it to perform syllogistic reasoning.
Our implementation uses a different approach to syllogisms than Boole, reducing the system of two equations to a single one.
However, the general procedure is still firmly grounded in Boole's system.